Import Data and Packages

In [24]:
# Setup relative imports
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from Helper.data import load_data

In [25]:
original = pd.read_csv('../Data/podcast_dataset.csv')

Filter Outliers

In [26]:
train, test = load_data()
train = train[(train["Number_of_Ads"] < 10) | (train["Number_of_Ads"].isna())]
train = train[(train["Episode_Length_minutes"] < 300) | (train["Episode_Length_minutes"].isna())]
train = train[(train["Host_Popularity_percentage"] < 100) | (train["Host_Popularity_percentage"].isna())]
train = train[(train["Guest_Popularity_percentage"] < 100) | (train["Guest_Popularity_percentage"].isna())]

Feature Engineering

In [27]:
TARGET = 'Listening_Time_minutes'

CATS = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

NUMS = ['Episode_Length_minutes', 'Host_Popularity_percentage', 
        'Guest_Popularity_percentage', 'Number_of_Ads']
FEATURES = NUMS + CATS

In [28]:
train_counts = train.nunique().reset_index()
train_counts.columns = ['Column', 'Train Unique']
test_counts = test.nunique().reset_index()
test_counts.columns = ['Column', 'Test Unique']
original_counts = original.nunique().reset_index()
original_counts.columns = ['Column', 'Original Unique']
merged_counts = pd.merge(train_counts, test_counts, on='Column', how='outer')
merged_counts = pd.merge(merged_counts, original_counts, on='Column', how='outer')
merged_counts

,Column,Train Unique,Test Unique,Original Unique
0,Episode_Length_minutes,12267,11631.0,11297.0
1,Episode_Sentiment,3,3.0,3.0
2,Episode_Title,100,100.0,100.0
3,Genre,10,10.0,10.0
4,Guest_Popularity_percentage,10003,9961.0,9899.0
5,Host_Popularity_percentage,8019,8010.0,7976.0
6,Listening_Time_minutes,42807,NaN,42909.0
7,Number_of_Ads,4,6.0,4.0
8,Podcast_Name,48,48.0,48.0
9,Publication_Day,7,7.0,7.0


Target Encoding

In [29]:
m = train[TARGET].mean()

ORIG_TARGET = []

for c in FEATURES:
    n = f"{c}2"
    ORIG_TARGET.append(n)  
    
    # Get target encoding mapping from original data
    target_mapping = original.groupby(c)[TARGET].mean()
    
    # Apply mapping to train and test
    if train[c].dtype.name == 'category':
        # For categorical columns, convert to string first
        train[n] = train[c].astype(str).map(target_mapping)
        test[n] = test[c].astype(str).map(target_mapping)
    else:
        # For non-categorical columns
        train[n] = train[c].map(target_mapping)
        test[n] = test[c].map(target_mapping)
    
    # Fill NA values
    train[n] = train[n].fillna(m)
    test[n] = test[n].fillna(m)

train.head(3)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,...,Episode_Length_minutes2,Host_Popularity_percentage2,Guest_Popularity_percentage2,Number_of_Ads2,Podcast_Name2,Episode_Title2,Genre2,Publication_Day2,Publication_Time2,Episode_Sentiment2
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,...,45.436281,43.420073,45.436281,48.525459,46.143074,43.525145,46.551083,45.545049,45.902440,46.940936
1,1,Joke Junction,Episode 26,119.8,Comedy,66.95,Saturday,Afternoon,75.95,2.0,...,87.105517,43.957970,36.098898,44.321965,43.651926,45.260247,44.524182,45.194999,45.937834,44.654776
2,2,Study Sessions,Episode 16,73.9,Education,69.97,Tuesday,Evening,8.97,0.0,...,59.046850,39.812068,36.644250,48.525459,45.938614,44.309041,45.222056,46.273659,44.989281,44.654776


In [30]:
for col in CATS:
    combined = pd.concat([train[col], test[col]], axis=0)
    codes, uniques = pd.factorize(combined)
    train[col] = codes[:len(train)]
    test[col] = codes[len(train):]
    
# Define the selected interaction features based on my local forward selection:
SELECTED_INTERACT = [
    'Episode_Length_minutes_Host_Popularity_percentage', 
    'Episode_Length_minutes_Guest_Popularity_percentage', 
    'Episode_Length_minutes_Number_of_Ads', 
    'Episode_Length_minutes_Publication_Time', 
    'Episode_Length_minutes_Episode_Sentiment', 
    'Host_Popularity_percentage_Guest_Popularity_percentage', 
    'Host_Popularity_percentage_Number_of_Ads', 
    'Host_Popularity_percentage_Podcast_Name', 
    'Host_Popularity_percentage_Publication_Time', 
    'Host_Popularity_percentage_Episode_Sentiment', 
    'Guest_Popularity_percentage_Number_of_Ads', 
    'Guest_Popularity_percentage_Publication_Day', 
    'Guest_Popularity_percentage_Publication_Time', 
    'Guest_Popularity_percentage_Episode_Sentiment', 
    'Episode_Title_Episode_Sentiment'
]

# Create the selected interaction features for both train and test.
for candidate in SELECTED_INTERACT:
    c1, c2 = None, None
    for f in FEATURES:
        prefix = f + '_'
        if candidate.startswith(prefix):
            possible_c2 = candidate[len(prefix):]
            if possible_c2 in FEATURES:
                c1 = f
                c2 = possible_c2
                break
    if c1 is None or c2 is None:
        raise ValueError(f"Unable to parse the candidate feature '{candidate}' into two base features.")
    
    # Create the interaction feature as the product of the two columns.
    train[candidate] = train[c1] * train[c2]
    test[candidate] = test[c1] * test[c2]

print("Selected interaction features have been created in both train and test.")

Selected interaction features have been created in both train and test.


Structure

In [31]:

train.describe()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,...,Host_Popularity_percentage_Guest_Popularity_percentage,Host_Popularity_percentage_Number_of_Ads,Host_Popularity_percentage_Podcast_Name,Host_Popularity_percentage_Publication_Time,Host_Popularity_percentage_Episode_Sentiment,Guest_Popularity_percentage_Number_of_Ads,Guest_Popularity_percentage_Publication_Day,Guest_Popularity_percentage_Publication_Time,Guest_Popularity_percentage_Episode_Sentiment,Episode_Title_Episode_Sentiment
count,749944.000000,749944.000000,749944.000000,662857.000000,749944.000000,749944.000000,749944.000000,749944.000000,603916.000000,749943.000000,...,603916.000000,749943.000000,749944.000000,749944.000000,749944.000000,603915.000000,603916.000000,603916.000000,603916.000000,749944.000000
mean,375002.479050,23.539920,49.436726,64.503447,4.556028,59.858266,3.050962,1.472987,52.234551,1.347929,...,3132.481171,80.232223,1410.087700,88.215689,59.998595,69.172635,160.039133,76.669953,52.223871,49.732537
std,216506.838496,13.917803,28.687858,32.967747,2.965915,22.871467,1.990350,1.117420,28.449797,1.110975,...,2201.614470,76.534991,1041.301654,79.123382,57.094785,75.903942,147.563009,78.460666,56.240624,54.998111
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,187502.750000,12.000000,24.000000,35.730000,2.000000,39.410000,1.000000,0.000000,28.380000,0.000000,...,1372.643700,0.000000,574.720000,0.000000,0.000000,0.000000,36.480000,0.000000,0.000000,0.000000
50%,375004.500000,23.000000,50.000000,63.840000,5.000000,60.050000,3.000000,1.000000,53.580000,1.000000,...,2672.889800,67.710000,1180.360000,75.320000,54.290000,48.380000,118.620000,57.750000,37.710000,33.000000
75%,562503.250000,36.000000,74.000000,94.070000,7.000000,79.530000,5.000000,2.000000,76.600000,2.000000,...,4563.426125,125.880000,2064.600000,142.060000,93.230000,104.280000,255.810000,126.270000,86.420000,83.000000
max,749999.000000,47.000000,99.000000,120.930000,9.000000,99.990000,6.000000,3.000000,99.990000,3.000000,...,9992.001600,299.970000,4699.060000,299.970000,199.980000,299.970000,599.940000,299.970000,199.980000,198.000000


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749944 entries, 0 to 749999
Data columns (total 37 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   id                                                      749944 non-null  int64  
 1   Podcast_Name                                            749944 non-null  int64  
 2   Episode_Title                                           749944 non-null  int64  
 3   Episode_Length_minutes                                  662857 non-null  float64
 4   Genre                                                   749944 non-null  int64  
 5   Host_Popularity_percentage                              749944 non-null  float64
 6   Publication_Day                                         749944 non-null  int64  
 7   Publication_Time                                        749944 non-null  int64  
 8   Guest_Popularity_percentage  

Training Model

In [33]:
np.random.seed(42)

features = [col for col in train.columns if col not in ['id', TARGET]]
X = train[features]
y = train['Listening_Time_minutes']

model = XGBRegressor(
    device='cuda',
    enable_categorical=True,
    n_estimators=5000,
    learning_rate=0.02,
    max_depth=11,
    subsample=0.8,
    colsample_bynode=0.6,
    random_state=42,
    verbosity=0,  # Standard-Regressionsziel: quadratischer Fehler
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kf)

rmse_scores = np.sqrt(-cv_scores)

print("RMSE pro Fold:", rmse_scores)
print("Durchschnittlicher RMSE:", rmse_scores.mean())


KeyboardInterrupt: 

In [22]:
model.fit(X, y)
test_features = test[features]
y_pred = model.predict(test_features)

In [23]:
result_df = pd.DataFrame({'id': test['id'], 'Listening_Time_minutes': y_pred})
model.save_model("../Submission/xgb_model.json")
result_df.to_csv('../Submission/best_model.csv', index=False)

In [161]:
result_df

,id,Listening_Time_minutes
0,750000,51.608578
1,750001,20.438503
2,750002,49.912762
3,750003,74.182564
4,750004,47.229687
...,...,...
249995,999995,10.843289
249996,999996,59.274998
249997,999997,8.204661
249998,999998,74.150032
